<a href="https://colab.research.google.com/github/caggursoy/phdSideQuests/blob/master/classifyAnimals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import tensorflow as tf

# define the model here
def build(width, height, depth, classes):
		# initialize the model and input shape
		model = Sequential()
		inputShape = (height, width, depth)

		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)

		model.add(Conv2D(20, (2,2), input_shape = inputShape))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
		model.add(Conv2D(100, (2,2), input_shape = inputShape))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
		model.add(Flatten())
		model.add(Dense(classes))
		model.add(Activation('sigmoid'))

		return model

# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 100
INIT_LR = 1e-3  #1e-3
BS = 32
trgIm = [75,75,3]
noClass = 10

# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("/content/drive/My Drive/Colab Notebooks/images/raw-img")))
random.seed(42)
random.shuffle(imagePaths)

# def labeler
def switch_label(argument):
	switcher = {
		"butterfly" : 0,
		"cat" : 1,
		"chicken" : 2,
		"cow" : 3,
		"dog" : 4,
		"elephant" : 5,
		"horse" : 6,
		"sheep" : 7,
		"spider" : 8,
		"squirrel" : 9
	}
	return switcher.get(argument)

# loop over the input images
idx=0
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	dims = image.shape
	if dims[0] != trgIm[0] or dims[1] != trgIm[1]:
		image = cv2.resize(image, (trgIm[0], trgIm[1]))
	image = img_to_array(image)
	data.append(image)
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(switch_label(label))
	# print(label, switch_label(label))
	print('\t', round(100*idx/len(imagePaths),2) , '%...', end="\r")
	idx=idx+1
# scale the raw pixel intensities to the range [0, 1]

# data = np.array(data, dtype="float") / 255.0
data = np.array(data, dtype="float") / np.amax(data)
labels = np.array(labels)

# partition the data into training and testing splits using 60% of
# the data for training and the remaining 20% for testing and 20% for validation
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

# validation split
(trainX, valX, trainY, valY) = train_test_split(trainX,
	trainY, test_size=0.25, random_state=42)

# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=noClass) # convert string to int
testY = to_categorical(testY, num_classes=noClass)
valY = to_categorical(valY, num_classes=noClass)

print("Training size: ",str(int(100*trainX.size/data.size)),"%")
print("Test size: ",str(int(100*testX.size/data.size)),"%")
print("Validation size: ",str(int(100*valX.size/data.size)),"%")

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

# initialize the model
print("[INFO] compiling model...")
model = build(width=trgIm[0], height=trgIm[1], depth=trgIm[2], classes=noClass)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# opt = SGD(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(valX, valY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1, shuffle=True, max_queue_size=10)
# H = model.fit(x=trainX, y=trainY, validation_data=(valX, valY), steps_per_epoch=None,
# 	epochs=EPOCHS, verbose=1, shuffle=False)

# save the model to disk
print("[INFO] serializing network...")
model.save("/content/drive/My Drive/Colab Notebooks/modelSaved.model")

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
# plt.show()
plt.savefig("accuracy.png")

[INFO] loading images...


KeyboardInterrupt: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.utils import plot_model
from keras import backend as K
from keras.models import Model
import tensorflow as tf
import numpy as np
import argparse
import imutils
import cv2
import os, random
from shutil import copyfile
import matplotlib.pyplot as plt
import pandas as pd

## Test part starts here
listIm = os.listdir("tests")

# define variables
faceAges = []
predictedList = []
score = 0;
range10 = 0;
range5 = 0;
outOf = 0;
idx = 0;
idx2 = 0;
layerNames = []
auxLayerNames = []
actvList = []
sumList = []
##
conv2d_1List = []
activation_1List = []
max_pooling2d_1List = []
conv2d_2List = []
activation_2List = []
max_pooling2d_2List = []
##
imNameList = []
activationsList = []
layerNamesList = []
resList = []

for imName in listIm:
  idx2 = 0;
  print(round((100*listIm.index(imName))/len(listIm),2),"%...",end="\r")
  auxName = imName
  imName = "tests/"+imName
  # print(imName)
  # load the image
  image = cv2.imread(imName)
  orig = image.copy()

  # pre-process the image for classification
  image = cv2.resize(image, (30, 30)) # 30,30
  image = image.astype("float") / 255.0
  image = img_to_array(image)
  image = np.expand_dims(image, axis=0)

  # load the trained convolutional neural network
  model = load_model("/content/drive/My Drive/Colab Notebooks/modelSaved.model")
  # Save model img to file
  plot_model(model, show_shapes=True, to_file= str('/content/drive/My Drive/Colab Notebooks/model_vis.png')

  # classify the input image
  results = model.predict(image)[0]
  resList.append(results)
  label = np.argmax(results)
  predictedList.append(label)
  imNameList.append(imName)
  idx2+=1

def switch_label(argument):
	switcher = {
		0 : "butterfly" ,
		1 : "cat" ,
		2 : "chicken",
		3 : "cow",
		4 : "dog",
		5 : "elephant",
		6 : "horse",
		7 : "sheep",
		8 : "spider",
		9 : "squirrel"
	}
	return switcher.get(argument)

for p in range(len(predictedList)):
    # print("Actual: ", imNameList[p], "\t","Final Prediction: ", switch_label(predictedList[p]), "\n", "Probs: ", resList[p])
    print("Actual: ", imNameList[p], "\tFinal Prediction: ", switch_label(predictedList[p]))